<a href="https://colab.research.google.com/github/addaia/TechnicalProject/blob/main/new_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets
!pip install sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.3 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
import re
import ast
import operator as op
import re
from sympy import Eq, solve
from sympy.parsing.sympy_parser import parse_expr, standard_transformations, implicit_multiplication_application

In [4]:
def evaluate_expression(expr):
    """
    evaluates an arithmetic expression given as a string or solves an equation if one is provided
    """


    if "=" in expr:
        # equation mode


        transformations = standard_transformations + (implicit_multiplication_application,)
        left_str, right_str = expr.split("=", 1)
        left_expr = parse_expr(left_str.strip(), transformations=transformations)
        right_expr = parse_expr(right_str.strip(), transformations=transformations)
        equation = Eq(left_expr, right_expr)

        free_symbols = list(equation.free_symbols)

        # if there is one free symbol, return its solution directly.
        if len(free_symbols) == 1:
            symbol = free_symbols[0]
            sol = solve(equation, symbol)
            return sol[0] if len(sol) == 1 else sol
        else:
            # for multiple variables, return the complete solution set.
            sol = solve(equation)
            return sol
    else:
        # arithmetic evaluation mode using AST
        operators = {
            ast.Add: op.add,
            ast.Sub: op.sub,
            ast.Mult: op.mul,
            ast.Div: op.truediv,
            ast.Pow: op.pow,
            ast.USub: op.neg,
            ast.Mod: op.mod,
        }
        def _eval(node):
            if isinstance(node, ast.Constant):
                return node.value
            elif isinstance(node, ast.Num):
                return node.n
            elif isinstance(node, ast.BinOp):
                left = _eval(node.left)
                right = _eval(node.right)
                return operators[type(node.op)](left, right)
            elif isinstance(node, ast.UnaryOp):
                return operators[type(node.op)](_eval(node.operand))
            else:
                raise TypeError(f"Unsupported expression: {node}")

        parsed_expr = ast.parse(expr, mode='eval').body
        return _eval(parsed_expr)

In [6]:
# load the GSM8K dataset
gsm8k = load_dataset("openai/gsm8k", "main")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [7]:
def add_code_column(example):
    answer = example["answer"]
    # extract all substrings between << and >>
    matches = re.findall(r"<<(.*?)>>", answer)
    if not matches:
        example["code"] = ""
        return example

    # process each consecutive pair of matches
    for i in range(len(matches) - 1):
        # split the previous match into its left-hand side (calculation) and right-hand side (result).
        parts_prev = matches[i].split("=")
        if len(parts_prev) != 2:
            continue  # skip to avoid errors
        prev_calc = parts_prev[0].strip()
        prev_result = parts_prev[1].strip()

        # split the next match
        parts_curr = matches[i+1].split("=")
        if len(parts_curr) != 2:
            continue
        curr_calc = parts_curr[0].strip()
        curr_result = parts_curr[1].strip()

        # wrap in parenthesis
        replacement = f"({prev_calc})"

        # replace
        new_curr_calc = re.sub(r'\b' + re.escape(prev_result) + r'\b', replacement, curr_calc)

        # reassamble
        matches[i+1] = new_curr_calc + "=" + curr_result

    # get string
    calc_part = matches[-1].strip().split('=')[0].strip()
    example["code"] = calc_part
    return example

gsm8k = gsm8k.map(add_code_column)


Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [8]:
gsm8k['train'][342]

{'question': 'In a 90-minute soccer game, Mark played 20 minutes, then rested after. He then played for another 35 minutes. How long was he on the sideline?',
 'answer': 'Mark played for a total of 20 + 35 = <<20+35=55>>55 minutes.\nSo, he was on the sideline for 90 - 55 = <<90-55=35>>35 minutes.\n#### 35',
 'code': '90-(20+35)'}

SOME ERROS ARE STILL


In [10]:
wrong_count = 0

def check_matches(example, idx):

    global wrong_count
    try:
        code_expr = example["code"]
        evaluated_value = evaluate_expression(code_expr)

        # extract answwer.
        match = re.search(r"####\s*(\S+)", example["answer"])
        if not match:
            return example

        actual_answer_str = match.group(1)
        actual_answer_str = actual_answer_str.replace(',', '') # some answer are like 88,112
        try:
            actual_value = int(actual_answer_str)
        except ValueError:
            actual_value = float(actual_answer_str)

        # evaluate
        if isinstance(evaluated_value, float) and evaluated_value.is_integer():
            evaluated_value = int(evaluated_value)

        if evaluated_value != actual_value:
            wrong_count += 1
            print(f"{idx}:")
            #print(example["question"])
            print("Code expression:", code_expr, "evaluates to", evaluated_value, "but expected", actual_value)

    except Exception as e:
        print( idx, e)
    return example

# check all entries
gsm8k = gsm8k.map(check_matches, with_indices=True)

# print number of wronmg asnwers
print("Number of wrong questions:", wrong_count)


Parameter 'function'=<function check_matches at 0x79ae8470bec0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

193:
Code expression: 192 = Y + Y * 0.2 evaluates to 160.000000000000 but expected 160
289:
Code expression: x+3+2x+12+8x=301 evaluates to 26 but expected 220
303:
Code expression: 48/12 evaluates to 4 but expected 50
343:
Code expression: (((4*125)*7)/1000)*0.14 evaluates to 0.49000000000000005 but expected 49
375:
Code expression: X + X*0.8 = 36 evaluates to 20.0000000000000 but expected 20
497:
Code expression: 150/5 evaluates to 30 but expected 10000
525 invalid syntax (<unknown>, line 1)
529:
Code expression: 50/2 evaluates to 25 but expected 26
552:
Code expression: (400*500)*.8 evaluates to 160000 but expected 16000
566:
Code expression: 8*80 evaluates to 640 but expected 1280
581 <class 'ast.UAdd'>
593:
Code expression: 34-(6+18) evaluates to 10 but expected 20
596 <class 'ast.UAdd'>
601:
Code expression: 90-(40*1.5) evaluates to 30 but expected 7
609:
Code expression: 80-42 evaluates to 38 but expected 20
618 invalid syntax (<unknown>, line 0)
623:
Code expression: 18/(1.5) ev

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

13:
Code expression: (5*2)+2 evaluates to 12 but expected 18
14:
Code expression: 16-((20-(20*20/100))*25/100) evaluates to 12 but expected 60
24 invalid syntax (<unknown>, line 0)
29:
Code expression: (33*2)+33 evaluates to 99 but expected 104
30:
Code expression: (11/(7+11)*162)+10 evaluates to 109.00000000000001 but expected 109
34:
Code expression: 5+(40/2) evaluates to 25 but expected 23
43:
Code expression: 300/5 evaluates to 60 but expected 48
81:
Code expression: 9+28 evaluates to 37 but expected 17
88 invalid syntax (<unknown>, line 0)
98:
Code expression: 30-5 evaluates to 25 but expected 5
107:
Code expression: (7-1)-1 evaluates to 5 but expected 3
114:
Code expression: 1800/6 evaluates to 300 but expected 6
135:
Code expression: 20-5 evaluates to 15 but expected 40
136 invalid syntax (<unknown>, line 0)
140:
Code expression: 20-(12+4) evaluates to 4 but expected 20
141:
Code expression: 100 evaluates to 100 but expected 400
160:
Code expression: 17+7 evaluates to 24 but exp

In [9]:
def update_code(example, idx):
    if idx == 8:
        example["code"] = "184-(30+46+38+11+18)"

    if idx == 29:
        example["code"] = "2p/5 = (p + 600)/4"

    if idx == 23:
        example["code"] = "(75-(35+(2*10)))/4"

    if idx == 32:
        example["code"] = "23-(5+8+0)"

    if idx == 44:
        example["code"] = "1000 * (100 - 30 - 15 - 25)/100"

    if idx == 48:
        example["code"] = "(24/6) + 6"

    if idx == 53:
        example["code"] = "(10 * 4) - (4*1)"

    if idx == 71:
        example["code"] = "(3*12)+7 - 18"

    if idx == 85:
        example["code"] = "(2+5) -2"

    if idx == 109: # wrong explanation in dataset
        example["code"] = "100 = A + 2*A + 2*A"

    if idx == 123:
        example["code"] = "((6/12) - (12/48))*100"

    if idx == 135:
        example["code"] = "4 * x + 600 = 10000"

    if idx == 143:
        example["code"] = "15*((100-25)/25)"

    if idx == 150:
        example["code"] = "(8m) + m = 27"

    if idx == 167: # WRONG ANSWER
        example['answer'] = "The first movie was 2*60=<<2*60=120>>120 minutes\nSo this movie is 120*.6=<<120*.6=72>>72 minutes longer\nSo this movie is 192 minutes\nIt also cost 50*2=$<<50*2=100>>100 per minute to film\nSo it cost 192*100=$1920\n#### 19200"
        example["code"] = "((60*2)*1.6)*(50*2)"

    if idx == 184:
        example["code"] = "(((2000+500)-50)+1500)-200"

    if idx == 193:
        example["code"] = "192 = Y + Y * 0.2"
    if idx == 198:
        example["code"] = "(20-(6+3+6+(6*(20/60))))* (60/6)"
    if idx == 232:
        example["code"] = "(15)-1-4-2"
    if idx == 242:
        example["code"] = "((3-2)+(2-1.5)+(4-2.5))*12"
    if idx == 256:
        example["code"] = "((3+3/8) * 8 ) / (1/4)"
    if idx == 264:
        example["code"] = "(5*600+((600*2)*2))*4"
    if idx == 289:
        example["code"] = "x+3+2x+12+8x=301"
    if idx == 293:
        example["code"] = "6*(1+3+5)"
    if idx == 302:
        example["code"] = "((1*2)*(1+3/4))*2"
    if idx == 323:
        example["code"] = "(1+1+(22+36))/5"
    if idx == 330:
        example["code"] = "220*((100-5)/100)"
    if idx == 339:
        example["code"] = "44*(1/2)*(1/2)"
    if idx == 343:
        example["code"] = "(((4*125)*7)/1000)*0.14"
    if idx == 345:
        example["code"] = "(40*((100-10)/100))*30"
    if idx == 356:
        example["answer"] = "If James only missed one question in all five rounds of five questions, he correctly answered 5 * 5 - 1 = <<5*5-1=24>>24 questions\nBefore the bonus, James earned 24 correct answers * 2 points/answer = <<24*2=48>>48 points.\nSince missing one question would only disqualify him from receiving a bonus in one round, he would earn a bonus for 5-1=<<5-1=4>>4 rounds.\nHe should expect a bonus of 4 rounds* 4 points/round= <<4*4=16>>16 points.\nIncluding his bonus, James scored a total of 48 + 16 = <<48+16=64>>64 points.\n#### 64"
        example["code"] = "((5*5)-1)*2 + (5-1)*4"
    if idx == 359:
        example["code"] = "(30+27+25)*(240/60)"
    if idx == 365:
        example["code"] = "(8*5)+((8+2)*3)"

    if idx == 375:
        example["code"] = "X + X*0.8 = 36"

    if idx == 386:
        example["code"] = "60+(100*((100-20)/100))"

    if idx == 393:
        example["code"] = "W + W + 2W + 2W = 30"

    if idx == 394:
        example["code"] = "((4*28)*5)/10"

    if idx == 415:
        example["code"] = "((6+6)*2)*2 + 30"

    if idx == 455:
        example["code"] = "((((1*2)*(1*2))*2)*2)/2 "

    if idx == 464:
        example["code"] = "C + (C/2) = 30"

    if idx == 473:
        example["code"] = "J + (J-40) = 250"

    if idx == 492:
        example["code"] = "40 * ((100- (10+60))/100)"

    if idx == 515:
        example["code"] = "((((32-8)/3)*5)/100)*((100-40)/100)*100"


    return example

gsm8k["train"] = gsm8k["train"].map(update_code, with_indices=True)



Map:   0%|          | 0/7473 [00:00<?, ? examples/s]